In [324]:
import operator

In [415]:
class Matrix:
    
    def __init__(self, *args):

        multi_array = []

        for a in args:

            if isinstance(a, list):
                multi_array.append(a)
        
        self.multi_array = multi_array
        self.matrix = self.multi_array


    def _apply_op(self, other, is_n, f):
        my_matrix = self.multi_array
        result = [[] for x in my_matrix]

        inter_result = 0

        my_row_len = len(my_matrix[0])
        my_row_n = len(my_matrix)

        n = 0
        e = 0
        while True:
            
            if e == my_row_len:
                e = 0
                n += 1

            if n == my_row_n:
                break

            e1 = my_matrix[n][e]
            e2 = other.multi_array[n][e] if is_n is False else other

            inter_result = f(e1, e2)
            result[n].append(inter_result)

            e += 1

        return result
    
    def _is_scalar(self, e):
        return True if type(e) == int or type(e) == float else False
    

    def __add__(self, other):
        return self._apply_op(other, self._is_scalar(other), operator.add)


    def __sub__(self, other):
        return self._apply_op(other, self._is_scalar(other), operator.sub)


    def __mul__(self, other):        
        return self._apply_op(other, self._is_scalar(other), operator.mul)


    def __truediv__(self, other):
        return self._apply_op(other, self._is_scalar(other), operator.truediv)


    def __matmul__(self, other):
        other_matrix = other.multi_array if isinstance(other, Matrix) else other
        my_matrix = self.multi_array
        result = [[] for x in my_matrix]

        inter_result = 0

        my_row_len = len(my_matrix[0])
        my_row_n = len(my_matrix)

        i = 0
        n = 0
        e = 0
        while True:
            
            if e == my_row_len:
                result[n].append(inter_result)
                inter_result = 0
                e = 0
                if i == my_row_n - 1:
                    n += 1
                    i = 0
                else:
                    i += 1

            if n == my_row_n:
                break

            e1 = my_matrix[n][e]
            e2 = other_matrix[e][i]

            inter_result += e1 * e2
            e += 1

        return result
    
    
    def _transpose(self, result):
        my_matrix = self.multi_array
        r = result
        e = 0

        def transpose_el(e1, discard_this):
            nonlocal e
            nonlocal r
            nonlocal my_matrix
            if e == len(my_matrix[0]):
                e = 0

            r[e].append(e1)
            e += 1

            return 0

        return transpose_el


    def T(self):
        # A bit inefficient since we store both the real result and the result of _apply_op that is
        # discarded in memory but eh ¯\_(ツ)_/¯ at least it's DRY.

        result = [[] for x in self.multi_array[0]]
        transpose_el = self._transpose(result)
        self._apply_op(0, True, transpose_el)
        return Matrix(*result)
       
    def __repr__(self):
        return str(self.multi_array)
        


In [334]:
my_matrix = [["foo", "bar", "foobar"], [1, 2, 3,]]
other_matrix = [[3, 4, 5,], ["good", "bad", "terrible"]]
c_rows = list(zip(my_matrix, other_matrix))
print(c_rows)
print(c_rows[0][0])

[(['foo', 'bar', 'foobar'], [3, 4, 5]), ([1, 2, 3], ['good', 'bad', 'terrible'])]
['foo', 'bar', 'foobar']


In [417]:
m1 = Matrix([1, 2, 3], [4, 5, 6])
t1 = Matrix([1, 4],
            [2, 5],
            [3, 6])
m2 = Matrix([6, 5, 4], [3, 2, 1])
m3 = Matrix([6, 5, 4], [3, 2, 1], [10, 20, 30])
m4 = Matrix([1, 2, 7], [7, 3, 4], [7, 5, 6])
print(m1 + m2)
print(m1 - m2)
print(m1 * 3)
print(m1 * m2)
print(m1 / 3)
print(m1 / m2)
print(m1 @ m2.T())
print(m1.T())
print(m1)

[[7, 7, 7], [7, 7, 7]]
[[-5, -3, -1], [1, 3, 5]]
[[3, 6, 9], [12, 15, 18]]
[[6, 10, 12], [12, 10, 6]]
[[0.3333333333333333, 0.6666666666666666, 1.0], [1.3333333333333333, 1.6666666666666667, 2.0]]
[[0.16666666666666666, 0.4, 0.75], [1.3333333333333333, 2.5, 6.0]]
[[28, 10], [73, 28]]
[[1, 4], [2, 5], [3, 6]]
[[1, 2, 3], [4, 5, 6]]


In [337]:
%pip install -Uq numpy

Note: you may need to restart the kernel to use updated packages.


In [338]:
import numpy as np

In [339]:
class Test:
    array = [[1, 2, 3], [1, 2, 3]]
    
    def __repr__(self):
        return str(self.array)
    
t = Test()
t

[[1, 2, 3], [1, 2, 3]]

In [340]:
%pip install -q ipytest pytest

import ipytest
import pytest
ipytest.autoconfig()

Note: you may need to restart the kernel to use updated packages.


In [418]:
%%ipytest

matrix1 = Matrix([1, 2, 3], [4, 5, 6])
matrix2 = Matrix([6, 5, 4], [3, 2, 1])

@pytest.mark.parametrize("matrix1,matrix2", [[matrix1, matrix2]])
def test_matrix_add_matrix(matrix1: Matrix, matrix2: Matrix):
    assert (
        (matrix1 + matrix2 == (np.array(matrix1.matrix) + np.array(matrix2.matrix))).all()
    ), 'add section with matrices is not working right'

@pytest.mark.parametrize("matrix1,number", [[matrix1, 13.2]])
def test_matrix_add_number(matrix1: Matrix, number: int | float):
    assert (
        (matrix1 + number == np.array(matrix1.matrix) + number).all()
    ), 'add section with matrix and number is not working right'

@pytest.mark.parametrize("matrix1,matrix2", [[matrix1, matrix2]])
def test_matrix_sub_matrix(matrix1: Matrix, matrix2: Matrix):
    assert (
        (matrix1 - matrix2 == np.array(matrix1.matrix) - np.array(matrix2.matrix)).all()
    ), 'sub section with matrices is not working right'

@pytest.mark.parametrize("matrix1,number", [[matrix1, 12.2]])    
def test_matrix_sub_number(matrix1: Matrix, number: int | float):
    assert (
        (matrix1 - number == np.array(matrix1.matrix) - number).all()
    ), 'sub section with matrix and number is not working right'

@pytest.mark.parametrize("matrix1", [matrix1])    
def test_matrix_transpose(matrix1: Matrix):
    assert (
        (matrix1.T().matrix == np.array(matrix1.matrix).T).all()
    ), 'transpose section is not working right'
    
# -----------------------------------------

@pytest.mark.parametrize("matrix1,matrix2", [[matrix1, matrix2]])
def test_matrix_mul_matrix(matrix1: Matrix, matrix2: Matrix):
    assert (
        (matrix1 * matrix2 == np.array(matrix1.matrix) * np.array(matrix2.matrix)).all()
    ), 'mul section with matrices is not working right'

@pytest.mark.parametrize("matrix1,number", [[matrix1, 13]])
def test_matrix_mul_number(matrix1: Matrix, number: int | float):
    assert (
        (matrix1 * number == np.array(matrix1.matrix) * number).all()
    ), 'mul section with matrix and number is not working right'

@pytest.mark.parametrize("matrix1,matrix2", [[matrix1, matrix2]])    
def test_matrix_div_matrix(matrix1: Matrix, matrix2: Matrix):
    assert (
        (matrix1 / matrix2 == np.array(matrix1.matrix) / np.array(matrix2.matrix)).all()
    ), 'div section with matrices is not working right'

@pytest.mark.parametrize("matrix1,number", [[matrix1, 2]])    
def test_matrix_div_number(matrix1: Matrix, number: int | float):
    assert (
        (matrix1 / number == np.array(matrix1.matrix) / number).all()
    ), 'div section with matrix and number is not working right'

@pytest.mark.parametrize("matrix1,matrix2", [[matrix1, matrix2.T()]])    
def test_matrix_matmul_matrix(matrix1: Matrix, matrix2: Matrix):
    assert(
        (matrix1 @ matrix2 == np.array(matrix1.matrix) @ np.array(matrix2.matrix)).all()
    ), 'matmul section is not working right, and don\'t forget about T() if needed'
    
    

..........                                                                                   [100%]
10 passed in 0.04s
